In [9]:
from src.ssunet.datasets import BinomDataset, ValidationDataset
from src.ssunet.models import Bit2Bit
from src.ssunet.configs import MasterConfig
from src.ssunet.constants import DEFAULT_CONFIG_PATH

config = MasterConfig.from_config(DEFAULT_CONFIG_PATH)
config.copy_config(DEFAULT_CONFIG_PATH)

# Load data for training and validation, ground truth for validation metrics
data = config.path_config.load_data_only()
# validation_data = config.path_config.load_reference_and_ground_truth()
validation_data = config.path_config.load_reference_only()

# Load data configurations and disable augmentation for validation
data_config = config.data_config
validation_config = data_config.validation_config

# Create training and validation datasets
training_data = BinomDataset(data, data_config, config.split_params)
validation_data = ValidationDataset(validation_data, validation_config)

# Create model
model = Bit2Bit(config.model_config)

# Create data loaders
training_loader = config.loader_config.loader(training_data)
validation_loader = config.loader_config.loader(validation_data)

# Print the input size
print(f"input_size: {tuple(next(iter(training_loader))[1].shape)}")

# Train the model
trainer = config.trainer
trainer.fit(model, training_loader, validation_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                             | Params | Mode 
-------------------------------------------------------------------------
0 | psnr_metric | PeakSignalNoiseRatio             | 0      | train
1 | ssim_metric | StructuralSimilarityIndexMeasure | 0      | train
2 | down_convs  | ModuleList                       | 9.6 M  | train
3 | up_convs    | ModuleList                       | 5.1 M  | train
4 | conv_final  | Sequential                       | 33     | train
-------------------------------------------------------------------------
14.7 M    Trainable params
0         Non-trainable params
14.7 M    Total params
58.718    Total estimated model params size (MB)
100       Modules in train mode
0         Modules in eval mode


input_size: (4, 1, 32, 128, 128)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

SystemExit: 0

/home/argar/miniconda3/envs/bit2bit/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
trainer.save_checkpoint(config.train_config.default_root_dir / "model.ckpt")

In [ ]:
from src.tools.tools import clear_vram
clear_vram()

In [5]:
from src.tools.gpuinference import gpu_patch_inference
import numpy as np
from tifffile import imwrite

output = gpu_patch_inference(
    model,
    data.primary_data.astype(np.float32),
    min_overlap=48,
    initial_patch_depth=64,
    device=config.device,
)

imwrite(config.train_config.default_root_dir / "input.tif", data.primary_data)
imwrite(config.train_config.default_root_dir / "inference.tif", output)

Inference #: 100%|██████████| 225/225 [02:53<00:00,  1.30it/s, vram_usage=188773.5 GB]


In [ ]:
from src.tools.tools import group_metrics
from tifffile import imread


input_new = data.primary_data.astype(np.float32)[:512].copy()
output_new = output[:512].copy()
ground_truth = imread(config.path_config.ground_truth_file)[:512].astype(np.float32).copy()

group_metrics(input_new, output_new, ground_truth, config.train_config.default_root_dir)